# Loading in Required Libraries

In [0]:
# Standard libraries/functions
import pandas as pd
import numpy as np

# Pyspark libraries/functions
import pyspark.sql.functions as F
from pyspark.sql.functions import col, sum, isnan, when, count, year, month, dayofmonth, date_format, concat_ws, acos, cos, radians, sin, udf, concat
from pyspark.sql.types import IntegerType, DateType, DoubleType, StringType, FloatType
from pyspark.sql.window import Window

# from pyspark.ml.linalg import Vectors#, VectorUDT
# from pyspark.ml.linalg import VectorType
from pyspark.mllib.linalg import Vectors, VectorUDT
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder, Imputer
from pyspark.ml.functions import vector_to_array
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.stat import Correlation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import PCA
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics

# Sklearn libraries/functions
from sklearn.utils import shuffle
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from pyspark.ml.classification import LogisticRegression, GBTClassifier, RandomForestClassifier, LinearSVC
import time

import mlflow
import mlflow.spark
# from pyspark.mllib.linalg import Vectors, VectorUDT


In [0]:
## library settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# Setting Up Storage

In [0]:
## Place this cell in any team notebook that needs access to the team cloud storage.

# The following blob storage is accessible to team members only (read and write)
# access key is valid til TTL
# after that you will need to create a new SAS key and authenticate access again via DataBrick command line
blob_container  = "smsj-261"       # The name of your container created in https://portal.azure.com
storage_account = "smsj"  # The name of your Storage account created in https://portal.azure.com
secret_scope    = "smsjscope"           # The name of the scope created in your local computer using the Databricks CLI
secret_key      = "smsjkey"             # The name of the secret key created in your local computer using the Databricks CLI
team_blob_url        = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"  #points to the root of your team storage bucket

# the 261 course blob storage is mounted here on the DataBricks workspace.
mids261_mount_path      = "/mnt/mids-w261"

# SAS Token: Grant the team limited access to Azure Storage resources
spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)
import pandas as pd
pdf = pd.DataFrame([[1, 2, 3, "Jane"], [2, 2,2, None], [12, 12,12, "John"]], columns=["x", "y", "z", "a_string"])
df = spark.createDataFrame(pdf) # Create a Spark dataframe from a pandas DF

# The following can write the dataframe to the team's Cloud Storage  
# Navigate back to your Storage account in https://portal.azure.com, to inspect the partitions/files.
# df.write.parquet(f"{team_blob_url}/test")

# see what's in the blob storage root folder 
display(dbutils.fs.ls(f"{team_blob_url}"))

path,name,size,modificationTime
wasbs://smsj-261@smsj.blob.core.windows.net/test/,test/,0,1689534418000


# Loading the Data

In [0]:
# UPLOADING 3 MONTH DATASET
# df_test_3m = spark.sql("SELECT * FROM test_3m")

# df_train_3m = spark.sql("SELECT * FROM train_3m")

In [0]:
# import spark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ReadingParquet").getOrCreate()

# df = spark.read.parquet("/path/to/parquet/file.parquet")
# df.show()

# spark = spark.sql.SparkSession.builder.appName("Call Parquet File").getOrCreate()

# df_test_3m = spark.read.parquet("dbfs:/user/hive/warehouse/test_3m")
# df_train_3m = spark.read.parquet("dbfs:/user/hive/warehouse/train_3m")

df_test_3m = spark.read.parquet("dbfs:/user/hive/warehouse/test_3m")
df_train_3m = spark.read.parquet("dbfs:/user/hive/warehouse/train_3m")

df_test_12m = spark.read.parquet("dbfs:/user/hive/warehouse/test_12m")
df_train_12m = spark.read.parquet("dbfs:/user/hive/warehouse/train_12m")

df_test_60m = spark.read.parquet("dbfs:/user/hive/warehouse/test_60m")
df_train_60m = spark.read.parquet("dbfs:/user/hive/warehouse/train_60m")

# df = spark.read.parquet("s3://my-bucket/test_3m.parquet")

# df_test_3m.createOrReplaceTempView("test_3m")
# df_train_3m.createOrReplaceTempView("df_train_3m")

# df_test_12m.createOrReplaceTempView("test_12m")
# df_train_12m.createOrReplaceTempView("df_train_12m")

# Use the DataFrame in your ML Flow project

In [0]:
from pyspark.sql.types import StringType, DoubleType, IntegerType

def total_count(df_train, df_test, months):
    # Concatenate the train and test dataframes to get the combined dataframe
    df = df_train.union(df_test)

    # Identify numerical and categorical columns
    numerical_features = [f.name for f in df.schema.fields if isinstance(f.dataType, (DoubleType, IntegerType))]
    categorical_features = [f.name for f in df.schema.fields if isinstance(f.dataType, StringType)]

    # Get the counts
    total_rows = df.count()
    total_columns = len(df.columns)
    total_num_features = len(numerical_features)
    total_cat_features = len(categorical_features)

    # Print results
    print(months, "Total rows:", total_rows)
    print(months, "Total columns:", total_columns)
    print(months, "Total numerical features:", total_num_features)
    print(months, "Total categorical features:", total_cat_features)
    print("------------------------------------------")

# Sample usage
# Assume df_train and df_test are PySpark DataFrames you've defined earlier
# total_count(df_train, df_test, "August")


In [0]:
total_count (df_train_3m, df_test_3m, "3 months")
total_count (df_train_12m, df_test_12m, "12 months")
total_count (df_train_60m, df_test_60m, "60 months")


3 months Total rows: 1401363
3 months Total columns: 103
3 months Total numerical features: 41
3 months Total categorical features: 25
------------------------------------------
12 months Total rows: 5811854
12 months Total columns: 109
12 months Total numerical features: 42
12 months Total categorical features: 23
------------------------------------------
60 months Total rows: 12926912
60 months Total columns: 109
60 months Total numerical features: 42
60 months Total categorical features: 23
------------------------------------------


# Initializing Functions

## Modelling Functions

In [0]:

def build_model (model_name, param_dict):
    '''
    Description: Contains models and hyperparameters
    Inputs:
    Outputs: 
    '''
    # Build Logistic Regression model
    if model_name == 'log':
        model = LogisticRegression(featuresCol = "allFeatures", 
                                    regParam=param_dict['regParam'], 
                                    maxIter=param_dict['maxIter'], 
                                    elasticNetParam=param_dict['elasticNetParam'])
    # Build Gradient Boosted Tree model
    elif model_name == 'gbt':
        model = GBTClassifier(featuresCol = "allFeatures",
                              maxDepth=param_dict['maxDepth'], # Default: 5
                              maxIter=param_dict['maxIter'], # Default: 20
                              maxBins=param_dict['maxBins'], # Default: 32
                              stepSize=param_dict['stepSize']) # Default: 0.1
    # Build Support Vector Machine model
    elif model_name == 'svm':
        model = LinearSVC(featuresCol="scaledFeatures",
                          maxIter=10)
    # Build Random Forest model
    elif model_name == 'rf':
        model = RandomForestClassifier(featuresCol = "scaledFeatures",
                                       numTrees=70,
                                       maxDepth=3, 
                                       seed=42)
    return model


# IMPROVED VERSION TO TEST:
# from pyspark.ml.classification import (LogisticRegression, GBTClassifier, 
#     LinearSVC, 
#     RandomForestClassifier
# )

# def build_model(model_name, param_dict):
#     """
#     Description: Contains models and hyperparameters
#     Inputs:
#     Outputs: 
#     """
#     model_class_mapping = {
#         'log': LogisticRegression,
#         'gbt': GBTClassifier,
#         'svm': LinearSVC,
#         'rf': RandomForestClassifier
#     }

#     model_class = model_class_mapping.get(model_name)
#     if model_class is None:
#         raise ValueError(f"Model '{model_name}' is not supported.")

#     model = model_class(featuresCol="allFeatures", **param_dict)
#     return model
 

In [0]:
# Obtain parameters from provided parameter dictionary
def udf_grid_search_params(params_grid):
    '''
    Description:
    Input:
    Output:
    '''
    cleaned_dict_list = []

    for param_dict in test_output:
        cleaned_dict = {param.name: value for param, value in param_dict.items()}
        cleaned_dict_list.append(cleaned_dict)
    return cleaned_dict_list


# IMPROVED VERSION TO TEST:
# def udf_grid_search_params(params_grid):
#     """
#     Description: Extract parameters from the provided parameter dictionary
#     Input: params_grid - List of dictionaries representing different parameter combinations
#     Output: List of dictionaries with cleaned parameters
#     """
#     cleaned_dict_list = [{param.name: value for param, value in param_dict.items()} for param_dict in params_grid]
#     return cleaned_dict_list


In [0]:
# Create pipline
def build_model_pipeline(input_df, input_categoricals_columns, input_pca_columns, input_non_pca_columns, input_prediction_feature = 'model_delay'):
    '''
    Description:
    Input:
    Output:
    '''
    base = input_df

    # input the categorical columns
    categoricals_columns = input_categoricals_columns

    # impute the missing categorical data using the mode
    # flight time category was missing data
    # we can actuall swtich to the CRS_DEP_TIME to fix this issue, but this is a function that we might have needed anyways or might need in the future
    base = impute_categoricals(base,base,categoricals_columns)
    
    indexers = map(lambda c: StringIndexer(inputCol=c, outputCol=c+"_idx", handleInvalid = 'keep'), categoricals_columns)
    ohes = map(lambda c: OneHotEncoder(inputCol=c + "_idx", outputCol=c+"_class"),categoricals_columns)

    # Establish features columns
    categoricals = list(map(lambda c: c+"_class", categoricals_columns))

    # input the number columns to be reduced with PCA
    numerics_pca_columns = input_pca_columns

    # input the columns we decide not to reduce with PCA
    numerics_non_pca_columns = input_non_pca_columns

    # input the feature we are trying to predict
    prediction_feature = input_prediction_feature

    all_numerics = numerics_pca_columns + numerics_non_pca_columns

    # imputer should handle all numeric columns regardless of usage in pca or not
    imputers = Imputer(inputCols = all_numerics, outputCols = all_numerics)

    # grab only relevant columns, we need numerics, categorical, and predictor
    base = base[all_numerics +categoricals_columns + [prediction_feature]]

    # VectorAssembler
    assembler_numeric_pca = VectorAssembler( inputCols=numerics_pca_columns, outputCol='features_numeric_pca_pre_scale')
    assembler_numeric_non_pca = VectorAssembler( inputCols=numerics_non_pca_columns, outputCol='features_numeric_non_pca_pre_scale')

    scaler_non_pca = StandardScaler(inputCol="features_numeric_non_pca_pre_scale",
                            outputCol="features_numeric_non_pca_scaled",
                            withStd=True,
                            withMean=True)

    scaler_pca = StandardScaler(inputCol="features_numeric_pca_pre_scale",
                            outputCol="features_numeric_pca_scaled",
                            withStd=True,
                            withMean=True)

    pca = PCA(k=2, inputCol='features_numeric_pca_scaled', outputCol='dense_vect_pca_features')

    assemblerAll = VectorAssembler(inputCols= ["features_numeric_non_pca_scaled", "dense_vect_pca_features"] +categoricals , outputCol="allFeatures")
    print(assemblerAll.getInputCols)

    label = StringIndexer(inputCol="model_delay", outputCol="label")

    model_matrix_stages =   list(indexers) + list(ohes) + \
                            [imputers] + \
                            [assembler_numeric_non_pca] + \
                            [assembler_numeric_pca] + \
                            [scaler_non_pca] + [scaler_pca] + \
                            [pca] + \
                            [assemblerAll] + \
                            [label]
    return model_matrix_stages


# IMPROVED VERSION TO TEST:
# from pyspark.ml.feature import StringIndexer, OneHotEncoder, Imputer, VectorAssembler, StandardScaler, PCA
# from pyspark.ml import Pipeline

# def build_model_pipeline(input_df, input_categoricals_columns, input_pca_columns, input_non_pca_columns, input_prediction_feature='model_delay'):
#     '''
#     Description: Create a pipeline for building the model matrix
#     Input:
#     Output: List of stages for the model matrix pipeline
#     '''
#     base_df = input_df

#     # Input the categorical columns
#     categorical_columns = input_categoricals_columns

#     # Impute the missing categorical data using the mode
#     # Flight time category was missing data
#     # We can actually switch to the CRS_DEP_TIME to fix this issue, but this is a function that we might have needed anyways or might need in the future
#     base_df = impute_categoricals(base_df, base_df, categorical_columns)

#     # Create stages for StringIndexer and OneHotEncoder
#     indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_idx", handleInvalid='keep') for col in categorical_columns]
#     ohes = [OneHotEncoder(inputCol=f"{col}_idx", outputCol=f"{col}_class") for col in categorical_columns]

#     # Establish feature columns
#     categorical_features = [f"{col}_class" for col in categorical_columns]

#     # Input the numeric columns to be reduced with PCA
#     numeric_pca_columns = input_pca_columns

#     # Input the columns we decide not to reduce with PCA
#     numeric_non_pca_columns = input_non_pca_columns

#     # Input the feature we are trying to predict
#     prediction_feature = input_prediction_feature

#     all_numeric_columns = numeric_pca_columns + numeric_non_pca_columns

#     # Imputer should handle all numeric columns regardless of usage in PCA or not
#     imputer = Imputer(inputCols=all_numeric_columns, outputCols=all_numeric_columns)

#     # Grab only relevant columns - numerics, categorical, and predictor
#     base_df = base_df[all_numeric_columns + categorical_columns + [prediction_feature]]

#     # Create VectorAssembler for numeric PCA and non-PCA features
#     assembler_numeric_pca = VectorAssembler(inputCols=numeric_pca_columns, outputCol='features_numeric_pca_pre_scale')
#     assembler_numeric_non_pca = VectorAssembler(inputCols=numeric_non_pca_columns, outputCol='features_numeric_non_pca_pre_scale')

#     # Create StandardScaler for numeric PCA and non-PCA features
#     scaler_non_pca = StandardScaler(inputCol="features_numeric_non_pca_pre_scale", outputCol="features_numeric_non_pca_scaled", withStd=True, withMean=True)
#     scaler_pca = StandardScaler(inputCol="features_numeric_pca_pre_scale", outputCol="features_numeric_pca_scaled", withStd=True, withMean=True)

#     # Create PCA stage
#     pca = PCA(k=2, inputCol='features_numeric_pca_scaled', outputCol='dense_vect_pca_features')

#     # Create VectorAssembler for all features
#     assembler_all = VectorAssembler(inputCols=["features_numeric_non_pca_scaled", "dense_vect_pca_features"] + categorical_features, outputCol="allFeatures")

#     # Create StringIndexer for the label
#     label = StringIndexer(inputCol=prediction_feature, outputCol="label")

#     # Build the list of stages for the model matrix pipeline
#     model_matrix_stages = indexers + ohes + [imputer, assembler_numeric_non_pca, assembler_numeric_pca, scaler_non_pca, scaler_pca, pca, assembler_all, label]

#     return model_matrix_stages


In [0]:

## Run time series cross validation and perform hyperparameter tuning to select the best model
def tscv(dataset, model, pca_numerics, non_pca_numerics, categoricals, k=5):
    '''
    Description: Runs time series cross validation on the provided model
    Input: Dataset, built model, PCA numeric features, non-PCA numeric features, categorical features, number of folds (k)
    Output: Average F1 score of model
    '''

    # initialize variables 
    n = dataset.count()
    chunk_size = int(n/(k+1))
    scores_auc = []
    scores_accuracy = []
    scores_precision = []
    scores_recall = []
    scores_f1 = []

    # Assume that we are ordering by FL_DATE always 
    sort_dataset = dataset.withColumn("row_id", F.row_number().over(Window.partitionBy().orderBy("FL_DATE")))

    # Perform tscv and hyperparameter tuning 
    for i in range(k):
        train_df = sort_dataset.filter(F.col('row_id') <= chunk_size * (i+1)).cache()
        val_df = sort_dataset.filter((F.col('row_id') > chunk_size * (i+1)) & (F.col('row_id') <= chunk_size * (i+2))).cache()

        build_pipeline_matrix = build_model_pipeline(train_df, categoricals,pca_numerics,non_pca_numerics,'model_delay')

        pipeline = Pipeline (stages=build_pipeline_matrix + [model])

        #Train model with train_df
        pipeline_model = pipeline.fit(train_df)
        
        # impute null validation categoricals basked on mode of training categoricals
        val_df = impute_categoricals(train_df, val_df, categoricals)

        # Predict on validation set
        predictions = pipeline_model.transform(val_df).select("probability", "label", "prediction")
        predictions = predictions.rdd.map(extract).toDF(["p0", "p1", "label", "prediction"])

        valid_input_model = predictions

        # Create an evaluator
        evaluator_auc = BinaryClassificationEvaluator(labelCol='label')
        evaluator_auc.setRawPredictionCol('p1')

        # Compute the areaUnderROC on the test data
        areaUnderROC = evaluator_auc.evaluate(valid_input_model, {evaluator_auc.metricName: "areaUnderROC"})

        evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

        # Compute various evaluation metrics
        accuracy = evaluator.evaluate(valid_input_model, {evaluator.metricName: "accuracy"})
        precision = evaluator.evaluate(valid_input_model, {evaluator.metricName: "weightedPrecision"})
        recall = evaluator.evaluate(valid_input_model, {evaluator.metricName: "weightedRecall"})
        f1 = evaluator.evaluate(valid_input_model, {evaluator.metricName: "f1"})

        print("Accuracy = %g" % accuracy)
        print("Precision = %g" % precision)
        print("Recall = %g" % recall)
        print("F1 = %g" % f1)

        # Append the score to the scores list
        scores_auc.append(areaUnderROC)
        scores_accuracy.append(accuracy)
        scores_precision.append(precision)
        scores_recall.append(recall)
        scores_f1.append(f1)

    f1_score = np.mean(scores_f1)
    return f1_score


In [0]:

def get_hyperparameters(dataset, model_type, hyperparameter_list, k):
    '''
    Description:
    Input:
    Output:
    '''
    # Initialize variables
    best_score = 0
    best_param = None
    start_time = time.time()
    iterations = 0
    # Loop through all combinations of hyperparameters and pick the best set
    for parameters in hyperparameter_list:
        iterations = iterations+1
        new_model = build_model(model_type, parameters)
        avg_score = tscv(dataset = dataset, model = new_model, k=k)
        if avg_score > best_score:
            best_score = avg_score
            best_params = parameters
        print(parameters)
        print('best score:', best_score, '|best params:', best_params)
        print("iteration: " + str(iterations) + " --- %s seconds ---" % (time.time() - start_time))
    return best_score, best_params

In [0]:

## Run time series cross validation and perform hyperparameter tuning to select the best model
def single_model_run(dataset, model, pca_numerics, non_pca_numerics, categoricals, k=5):
    '''
    Description: Runs time series cross validation on the provided model
    Input: Dataset, built model, PCA numeric features, non-PCA numeric features, categorical features, number of folds (k)
    Output: Average F1 score of model
    '''
    # initialize variables 
    # n = dataset.count()
    # chunk_size = int(n/(k+1))
    scores_auc = []
    scores_accuracy = []
    scores_precision = []
    scores_recall = []
    scores_f1 = []

    # Assume that we are ordering by FL_DATE always 
    sort_dataset = dataset.withColumn("row_id", F.row_number().over(Window.partitionBy().orderBy("FL_DATE",F.rand())))

    model_n = sort_dataset.count()
    split_factor = 0.7

    # Perform tscv and hyperparameter tuning 

    train_df = sort_dataset.filter(F.col('row_id') <= (model_n * split_factor)).cache()
    val_df = sort_dataset.filter((F.col('row_id') > (model_n * split_factor))).cache()

    build_pipeline_matrix = build_model_pipeline(train_df, categoricals,pca_numerics,non_pca_numerics,'model_delay')

    pipeline = Pipeline (stages=build_pipeline_matrix + [model])

    #Train model with train_df
    pipeline_model = pipeline.fit(train_df)
    
    # impute null validation categoricals basked on mode of training categoricals
    val_df = impute_categoricals(train_df, val_df, categoricals)

    # Predict on validation set
    predictions = pipeline_model.transform(val_df).select("probability", "label", "prediction")
    predictions = predictions.rdd.map(extract).toDF(["p0", "p1", "label", "prediction"])

    valid_input_model = predictions

    # Create an evaluator
    evaluator_auc = BinaryClassificationEvaluator(labelCol='label')
    evaluator_auc.setRawPredictionCol('p1')

    # Compute the areaUnderROC on the test data
    areaUnderROC = evaluator_auc.evaluate(valid_input_model, {evaluator_auc.metricName: "areaUnderROC"})

    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

    # Compute various evaluation metrics
    accuracy = evaluator.evaluate(valid_input_model, {evaluator.metricName: "accuracy"})
    precision = evaluator.evaluate(valid_input_model, {evaluator.metricName: "weightedPrecision"})
    recall = evaluator.evaluate(valid_input_model, {evaluator.metricName: "weightedRecall"})
    f1 = evaluator.evaluate(valid_input_model, {evaluator.metricName: "f1"})

    print("Accuracy = %g" % accuracy)
    print("Precision = %g" % precision)
    print("Recall = %g" % recall)
    print("F1 = %g" % f1)

    # Append the score to the scores list
    scores_auc.append(areaUnderROC)
    scores_accuracy.append(accuracy)
    scores_precision.append(precision)
    scores_recall.append(recall)
    scores_f1.append(f1)

    f1_score = np.mean(scores_f1)

    return f1_score


In [0]:

def get_hyperparameters_single_run(dataset, model_type, hyperparameter_list, k_folds, pca_numerics, non_pca_numerics, categoricals):
    '''
    Description:
    Input:
    Output:
    '''
    # Initialize variables
    best_score = 0
    best_param = None

    # Loop through all combinations of hyperparameters and pick the best set
    for parameters in hyperparameter_list:
        new_model = build_model(model_type, parameters)
        avg_score = single_model_run(dataset = dataset, model = new_model, pca_numerics = pca_num, non_pca_numerics = non_pca_num, categoricals = categoricals, k = k_folds)
        if avg_score > best_score:
            best_score = avg_score
            best_params = parameters
        print(parameters)
        print('best score:', best_score, '|best params:', best_params)
    return best_score, best_params


In [0]:
## Run time series cross validation and perform hyperparameter tuning to select the best model
def single_model_fit(dataset, model, pca_numerics, non_pca_numerics, categoricals, k=5):
    '''
    Description: Runs time series cross validation on the provided model
    Input: Dataset, built model, PCA numeric features, non-PCA numeric features, categorical features, number of folds (k)
    Output: Average F1 score of model
    '''
    # initialize variables 
    # n = dataset.count()
    # chunk_size = int(n/(k+1))
    scores_auc = []
    scores_accuracy = []
    scores_precision = []
    scores_recall = []
    scores_f1 = []

    # Assume that we are ordering by FL_DATE always 
    sort_dataset = dataset.withColumn("row_id", F.row_number().over(Window.partitionBy().orderBy("FL_DATE",F.rand())))

    model_n = sort_dataset.count()
    split_factor = 0.7

    # Perform tscv and hyperparameter tuning 

    train_df = sort_dataset.filter(F.col('row_id') <= (model_n * split_factor)).cache()
    val_df = sort_dataset.filter((F.col('row_id') > (model_n * split_factor))).cache()

    build_pipeline_matrix = build_model_pipeline(train_df, categoricals,pca_numerics,non_pca_numerics,'model_delay')

    pipeline = Pipeline(stages=build_pipeline_matrix + [model])

    #Train model with train_df
    pipeline_model = pipeline.fit(train_df)

    #################### Feature Importances Code ######################

    ###################################################################
    return pipeline_model


In [0]:
def extract(row):
    '''
    Description:
    Input:
    Output:
    '''
    
    return tuple(row.probability.toArray().tolist()) +  (row.label,) + (row.prediction,)

# 4. Model Pipeline (Modelling & Hyperparameter Tuning)

## 4.1 Logistic Regression

# ML Flow Model

In [0]:
def RegressionEvaluator(preds):
    # print(preds)
    rdd_preds_m = preds.select(['prediction', 'label']).rdd

    # predictions = pipeline_model.transform(df_test_3m).select("probability", "label", "prediction")
    # predictions = predictions.rdd.map(extract).toDF(["p0", "p1", "label", "prediction"])

    preds = preds.select("probability", "label", "prediction")
    preds = preds.rdd.map(extract).toDF(["p0", "p1", "label", "prediction"])


    # Create an binary evaluator
    evaluator_auc = BinaryClassificationEvaluator(labelCol='label')
    evaluator_auc.setRawPredictionCol('p1')

    # Compute the areaUnderROC on the test data
    areaUnderROC = evaluator_auc.evaluate(preds, {evaluator_auc.metricName: "areaUnderROC"})
    areaUnderPR = evaluator_auc.evaluate(preds, {evaluator_auc.metricName: "areaUnderPR"})

    multi_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
    multi_evaluator2 = MulticlassMetrics(rdd_preds_m)
    # Compute various evaluation metrics
    accuracy = multi_evaluator.evaluate(preds, {multi_evaluator.metricName: "accuracy"})
    precision = multi_evaluator.evaluate(preds, {multi_evaluator.metricName: "precisionByLabel"})
    recall = multi_evaluator.evaluate(preds, {multi_evaluator.metricName: "recallByLabel"})
    f1 = multi_evaluator.evaluate(preds, {multi_evaluator.metricName: "f1"})

    f2 = np.round(multi_evaluator2.fMeasure(label=1.0, beta=2.0), 5)
    # pr = binary_evaluator.areaUnderPR

    return accuracy, precision, recall, f1, f2 ,areaUnderROC,areaUnderPR



In [0]:
df_train = df_train_12m
df_test = df_test_12m

In [0]:

# cat_features = ["REPORT_TYPE","DAY_OF_WEEK","flight_time_category", "DISTANCE_BIN", "CANCELLED", "us_holiday"]

# categoricals = list(map(lambda c: c+"_class", cat_features))

# assemblerAll = VectorAssembler(inputCols= ["numeric_scaled_non_pca", "dense_vect_pca_features"] +categoricals , outputCol="features")


# model_matrix_stages = [assemblerAll] #+ [label]

# lr = LogisticRegression()

# pipeline = Pipeline(stages=model_matrix_stages + [lr])

In [0]:
# CHANGES HERE
from pyspark.sql.functions import when, col

# Suppose `df` is your dataframe and `label` is your target column
numNegatives = df_train_12m.filter(col("label") == 0).count()
numPositives = df_train_12m.filter(col("label") == 1).count()
total = df.count()

# Calculate class weights
weightForPositive = total / (2.0 * numPositives)
weightForNegative = total / (2.0 * numNegatives)

# Add a new column 'classWeight' to df
df_train = df_train_12m.withColumn("classWeight", when(col("label") == 1, weightForPositive).otherwise(weightForNegative))

numNegatives = df_test_12m.filter(col("label") == 0).count()
numPositives = df_test_12m.filter(col("label") == 1).count()
total = df.count()

# Calculate class weights
weightForPositive = total / (2.0 * numPositives)
weightForNegative = total / (2.0 * numNegatives)

# Add a new column 'classWeight' to df
df_test = df_test_12m.withColumn("classWeight", when(col("label") == 1, weightForPositive).otherwise(weightForNegative))

lr = LogisticRegression(featuresCol="allFeatures", weightCol="classWeight")
pipeline = Pipeline(stages=[lr])


from hyperopt import hp
search_space = {
    "maxIter": hp.quniform("maxIter", 10,200,10),
    "regParam": hp.quniform("regParam", 0.1,0.5,0.01,),
    "elasticNetParam": hp.quniform("elasticNetParam", 0.0,1,0.1)
}
search_space

{'maxIter': <hyperopt.pyll.base.Apply at 0x7f9b141bd720>,
 'regParam': <hyperopt.pyll.base.Apply at 0x7f9b141bdc00>,
 'elasticNetParam': <hyperopt.pyll.base.Apply at 0x7f9b141bc850>}

In [0]:

def objective_function(params):
    # CHANGES HERE
    maxIter = params["maxIter"]
    regParam = params["regParam"]
    elasticNetParam = params["elasticNetParam"]
    start_time = time.time()

    with mlflow.start_run():
        # CHANGES HERE
        estimator = pipeline.copy({lr.maxIter:maxIter,
                                  lr.regParam:regParam,
                                  lr.elasticNetParam: elasticNetParam})
        model = estimator.fit(df_train)

        preds_training = model.transform(df_train)       
        pred_calc_training = RegressionEvaluator(preds_training)  

        train_accuracy = pred_calc_training[0]
        train_precision = pred_calc_training[1]
        train_recall = pred_calc_training[2]
        train_f1_score = pred_calc_training[3]
        train_f2_score = pred_calc_training[4]
        train_areaUnderROC = pred_calc_training[5]
        train_areaUnderPR = pred_calc_training[6]

        mlflow.log_metric('train_accuracy', train_accuracy)
        mlflow.log_metric('train_precision', train_precision)
        mlflow.log_metric('train_recall', train_recall)
        mlflow.log_metric('train_f1_score', train_f1_score)
        mlflow.log_metric('train_f2_score', train_f2_score)
        mlflow.log_metric('train_areaUnderROC', train_areaUnderROC)
        mlflow.log_metric('train_areaUnderPR', train_areaUnderPR)


        print('-------------------')
        print('Train Metrics:')
        print('accuracy:',train_accuracy)
        print('precision:',train_precision)
        print('recall:',train_recall)

        print('f1_score:',train_f1_score)
        print('f2_score:',train_f2_score)

        print('areaUnderROC:',str(train_areaUnderROC))
        print('areaUnderPR:',str(train_areaUnderPR))

        preds = model.transform(df_test)
        pred_calc = RegressionEvaluator(preds)
        val_accuracy = pred_calc[0]
        val_precision = pred_calc[1]
        val_recall = pred_calc[2]
        val_f1_score = pred_calc[3]
        val_f2_score = pred_calc[4]
        val_areaUnderROC = pred_calc[5]
        val_areaUnderPR = pred_calc[6]

        mlflow.log_metric('val_accuracy', val_accuracy)
        mlflow.log_metric('val_precision', val_precision)
        mlflow.log_metric('val_recall', val_recall)
        mlflow.log_metric('val_f1_score', val_f1_score)
        mlflow.log_metric('val_f2_score', val_f2_score)
        mlflow.log_metric('val_areaUnderROC', val_areaUnderROC)
        mlflow.log_metric('val_areaUnderPR', val_areaUnderPR)
        print('-------------------')
        print('Validation Metrics:')
        print('accuracy:',val_accuracy)
        print('precision:',val_precision)
        print('recall:',val_recall)
        print('f1_score:',val_f1_score)
        print('f2_score:',val_f2_score)
        print('areaUnderROC:',val_areaUnderROC)
        print('areaUnderPR:',val_areaUnderPR)

        print('-------------------')
        print('Model Params:')
        print('maxIter:',maxIter)
        print('regParam:',regParam)
        print('elasticNetParam:',elasticNetParam)
 

        mlflow.spark.log_model(model, "XGB_Model_test_JT_3m_fast_test")
        print('model_logging_complete' + " --- %s seconds ---" % (time.time() - start_time))
    return val_recall


In [0]:
from hyperopt import fmin, tpe, Trials
import mlflow

start_time = time.time()
mlflow.pyspark.ml.autolog(log_models=False)
num_evals = 10
trials = Trials()
best_hyperparam = fmin(fn=objective_function,
                       space = search_space,
                       algo=tpe.suggest,
                       max_evals = num_evals,
                       trials=trials,
                       rstate=np.random.default_rng(42))


  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2023/08/08 20:40:00 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.

/databricks/spark/python/pyspark/sql/context.py:165: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(



-------------------
Train Metrics:
accuracy:
0.8034807544741522
precision:
0.8034807544741522
recall:
1.0
f1_score:
0.7159281530548826
f2_score:
0.0
areaUnderROC:
0.5
areaUnderPR:
0.19651924552584774
-------------------
Validation Metrics:
accuracy:
0.8490596592692387
precision:
0.8490596592692387
recall:
1.0
f1_score:
0.7797501842458682
f2_score:
0.0
areaUnderROC:
0.5
areaUnderPR:
0.1509403407307613
-------------------
Model Params:
maxIter:
130.0
regParam:
0.43
elasticNetParam:
0.9
  0%|          | 0/10 [02:48<?, ?trial/s, best loss=?]

2023/08/08 20:42:54 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().

/databricks/python/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



model_logging_complete --- 217.94420051574707 seconds ---
 10%|█         | 1/10 [03:38<32:43, 218.15s/trial, best loss: 1.0]

...

model_logging_complete --- 274.498642206192 seconds ---
 90%|█████████ | 9/10 [39:41<04:34, 274.41s/trial, best loss: 0.5075459554026953]

2023/08/08 21:19:42 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.



-------------------
Train Metrics:
accuracy:
0.730173986652554
precision:
0.8451982242636832
recall:
0.8131017967497374
f1_score:
0.7372817527113507
f2_score:
0.37934
areaUnderROC:
0.6432970619979722
areaUnderPR:
0.2963860528261424
-------------------
Validation Metrics:
accuracy:
0.7382290588047258
precision:
0.8764188159369899
recall:
0.8052371887743818
f1_score:
0.7570276923557908
f2_score:
0.33105
areaUnderROC:
0.6246518773015473
areaUnderPR:
0.2187062673557174
-------------------
Model Params:
maxIter:
100.0
regParam:
0.23
elasticNetParam:
0.4
 90%|█████████ | 9/10 [43:31<04:34, 274.41s/trial, best loss: 0.5075459554026953]

2023/08/08 21:23:36 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().



model_logging_complete --- 280.3648474216461 seconds ---
100%|██████████| 10/10 [44:22<00:00, 266.21s/trial, best loss: 0.5075459554026953]


In [0]:
df_test_12m.count()

3734871

In [0]:
df_train_12m.count()

9192041

In [0]:
df_test_12m.count() + df_train_12m.count()

12926912